In [1]:
import numpy as np 
import tensorflow as tf 
import pandas as pd 
import json as js 
import os
import openpyxl as xl 
import itertools as it

In [2]:
def transform_strings(strings_list):

    result_STE_labels = strings_list
    allowed_simbols = "ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮйцукенгшщзхъфывапролджэячсмитьбюQWERTYUIOPASDFGHJKLZXCVBNMqwertyuiopasdfghjklzxcvbnm12345467890"

    for (label_number, STE_label) in enumerate(strings_list):

        tmp_STE_label = ""
        if (type(STE_label) != float):

            for simbol in STE_label:
                        
                if (simbol in allowed_simbols):
                    tmp_STE_label += simbol.lower()

                else:
                    tmp_STE_label += " "
                
            tmp_STE_list = tmp_STE_label.split()
            result_STE_list = []
            iterator = 0
            while iterator < len(tmp_STE_list):

                sub_string = tmp_STE_list[iterator]
                delete_condition = False
                if len(sub_string) >= 4:

                    for simbol in sub_string:

                        if (simbol in "1234567890"):

                            delete_condition = True
                            break

                    if not delete_condition:
                        result_STE_list.append(sub_string)
                
                iterator += 1
            
            result_STE_label = " ".join(sub_string for sub_string in result_STE_list)     
            result_STE_labels[label_number] = result_STE_label
                
        else:
            result_STE_labels[label_number] = STE_label
    
    return result_STE_labels

def sort_strings_list(strings_list):

    result_strings_list = []
    cn_collection = {}
    
    for string in strings_list:
        
        if (type(string) != float):

            tmp_string_list = string.split()
            done_condition = False
            curent_list_part = 1
            while curent_list_part < len(tmp_string_list):

                for sub_string_number in range(len(tmp_string_list) - curent_list_part):

                    if (tmp_string_list[sub_string_number] > tmp_string_list[sub_string_number + 1]):
                        
                        tmp_string_list[sub_string_number], tmp_string_list[sub_string_number + 1] = \
                        tmp_string_list[sub_string_number + 1], tmp_string_list[sub_string_number]

                curent_list_part += 1

            result_string = " ".join(sub_string for sub_string in tmp_string_list)
            result_strings_list.append(result_string)
            cn_collection[result_string] = string
        
        else:
            result_strings_list.append(string)
            cn_collection[string] = string
    
    return (result_strings_list, cn_collection)

Консолидация данных

In [3]:


def collect_data_turnover(data_path):
    
    indexator = 1
    general_STE_labels = []
    data_collection = {
        "Оборотная ведомость сч. 105": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
        "Оборотно-Сальдовая ведомость сч. 21": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
        "Оборотно-Сальдовая ведомость сч. 101": {
            "квартал_1":{},
            "квартал_2": {},
            "квартал_3": {},
            "квартал_4": {}
        },
    }

    general_STE_labels = []
    for data_batch_f in os.listdir(data_path):
        
        if indexator > 4:
        
            general_STE_labels = []
            indexator = 1

        curent_batch_f = os.path.join(data_path, data_batch_f)
        curent_dataframe = pd.read_excel(curent_batch_f)

        if "сч. 105" in data_batch_f:

            report_type = "Оборотная ведомость " + "сч. 105"
            STE_labels = curent_dataframe["Наименование нефинансового актива"].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_dataframe["Наименование нефинансового актива"] = STE_labels
            curent_dataframe.columns = ["№ п/п", "Инвентарный номенклатурный номер", "Код справочника", "Наименование нефинансового актива",
                                       "Единица измерения", "Остаток на 1 января 2022 г.|дебет|кол-во", 
                                       "Остаток на 1 января 2022 г.|дебет|сумма", "Оборот за 1 квартал 2022 г.|дебет|кол-во", 
                                       "Оборот за 1 квартал 2022 г.|дебет|сумма", "Оборот за 1 квартал 2022 г.|кредит|кол-во", 
                                       "Оборот за 1 квартал 2022 г.|кредит|сумма", "Остаток на 31 марта 2022 г.|дебет|кол-во", 
                                       "Остаток на 31 марта 2022 г.|дебет|сумма"]
        
        elif ("сч. 101" in data_batch_f) or ("сч. 21"  in data_batch_f):

            report_type = "Оборотно-Сальдовая ведомость " + data_batch_f[data_batch_f.find("сч"): data_batch_f.find("за") - 1]
            STE_labels_index = curent_dataframe.columns[0]
            STE_labels = curent_dataframe[STE_labels_index].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_dataframe[STE_labels_index] = STE_labels

        general_STE_labels += STE_labels
        
        data_collection[report_type]["general_STE_labels"] = general_STE_labels
        data_collection[report_type][f"квартал_{indexator}"] = {
            "data": curent_dataframe,
            "STE_labels": STE_labels
        }
        general_STE_labels += STE_labels

        indexator += 1

    return data_collection

data_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Обороты по счету"
data_collection = collect_data_turnover(data_path=data_path)
print(data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"].keys())

print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_2"]["STE_labels"]))
print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_3"]["STE_labels"]))
print(len(data_collection["Оборотная ведомость сч. 105"]["квартал_4"]["STE_labels"]))
# data_collection["Оборотная ведомость сч. 105"]["квартал_1"]["data"].iloc[:100].tail()
# data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"]["data"].iloc[:50].tail()
# data_collection["Оборотно-Сальдовая ведомость сч. 21"]["квартал_1"]["data"].iloc[:100].tail()





dict_keys(['data', 'STE_labels'])
212
209
260


Приведение таблицы ведомостей 101 и 21 к нормальному формату

In [4]:
for data_batch in data_collection.keys():

    if data_batch != "Оборотная ведомость сч. 105":

        curent_data_batch = data_collection[data_batch]
        for time_kvartal in curent_data_batch.keys():

            if time_kvartal != "general_STE_labels":

                curent_dataframe = curent_data_batch[time_kvartal]["data"]
                for column in curent_dataframe.columns:

                    if (column in ['ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ "ИНФОРМАЦИОННЫЙ ГОРОД"', 
                        "Unnamed: 4", "Unnamed: 10", "Unnamed: 14", "Unnamed: 9"]) == False:
                        del curent_dataframe[column]

                curent_dataframe.columns = ["СТЕ", "Код", "Определитель", "Сальдо на начало периода|дебет", "Сальдо на конец периода|дебет"]
                amount_dataframe = curent_dataframe[curent_dataframe["Определитель"] == "Сумма"]
                count_dataframe = curent_dataframe[curent_dataframe["Определитель"] == "Кол."]

                del amount_dataframe["Определитель"]
                amount_dataframe.columns = ["СТЕ", "Код", "Сальдо на начало периода|дебет|сумма", "Сальдо на конец периода|дебет|сумма"]
                
                del count_dataframe["Определитель"]
                count_dataframe.columns = ["СТЕ", "Код", "Сальдо на начало периода|дебет|кол.", "Сальдо на конец периода|дебет|кол."]
                count_dataframe["СТЕ"] = amount_dataframe["СТЕ"].to_list()
                count_dataframe["Код"] = amount_dataframe["Код"].to_list()


                
                data_collection[data_batch][time_kvartal]["data"] = [amount_dataframe, count_dataframe]




C:\Users\1\AppData\Local\Temp\ipykernel_12980\497594706.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_dataframe["СТЕ"] = amount_dataframe["СТЕ"].to_list()
C:\Users\1\AppData\Local\Temp\ipykernel_12980\497594706.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_dataframe["Код"] = amount_dataframe["Код"].to_list()


In [11]:
data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"]["data"][0].tail()

,СТЕ,Код,Сальдо на начало периода|дебет|сумма,Сальдо на конец периода|дебет|сумма
7651,valberg офисный сейф трейзером,NaN,35254.35,35254.35
7653,кузнецов,02-0000000017,35254.35,35254.35
7655,brandmauer взломостойкий металлический огнесто...,NaN,64395.65,64395.65
7657,кузнецов,02-0000000017,64395.65,64395.65
7659,итого,NaN,41670896.38,41712447.58


In [14]:
data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"]["data"][1].tail()

,СТЕ,Код,Сальдо на начало периода|дебет|кол.,Сальдо на конец периода|дебет|кол.
7652,valberg офисный сейф трейзером,NaN,1,1
7654,кузнецов,02-0000000017,1,1
7656,brandmauer взломостойкий металлический огнесто...,NaN,1,1
7658,кузнецов,02-0000000017,1,1
7660,итого,NaN,2990,2991


In [16]:
data_collection["Оборотно-Сальдовая ведомость сч. 101"]["квартал_1"]["data"][0].tail()

,СТЕ,Код,Сальдо на начало периода|дебет|сумма,Сальдо на конец периода|дебет|сумма
7651,valberg офисный сейф трейзером,NaN,35254.35,35254.35
7653,кузнецов,02-0000000017,35254.35,35254.35
7655,brandmauer взломостойкий металлический огнесто...,NaN,64395.65,64395.65
7657,кузнецов,02-0000000017,64395.65,64395.65
7659,итого,NaN,41670896.38,41712447.58


In [5]:
reminder_data_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Складские остатки"
def collect_data_reminder(data_path):

    data_collection = {
        "Ведомость остатков (сч. 105)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
        "Ведомость остатков (сч. 101)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
        "Ведомость остатков (сч. 21)": {
            "31.03.2022": {},
            "30.06.2022": {},
            "30.09.2022": {},
            "31.12.2022": {}
        },
    }

    for reminder_sheet_f in os.listdir(data_path):
        
        curent_sample_f = os.path.join(data_path, reminder_sheet_f)
        curent_sample_df = pd.read_excel(curent_sample_f)

        if ("г." not in reminder_sheet_f):
            curent_sample_date = reminder_sheet_f[reminder_sheet_f.find("3"): reminder_sheet_f.find("(") - 1]

        else:
            curent_sample_date = reminder_sheet_f[reminder_sheet_f.find("3"): reminder_sheet_f.find("г.")]

        if "(сч. 105)" in reminder_sheet_f:
            
            report_type = "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]
            STE_labels = curent_sample_df["МОЛ"].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            curent_sample_df["МОЛ"] = STE_labels

        
        elif ("(сч. 21)" in reminder_sheet_f) or ("(сч. 101)" in reminder_sheet_f):
            
            
            report_type = "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]
            "Ведомость остатков " + reminder_sheet_f[reminder_sheet_f.find("(сч."): reminder_sheet_f.find(")") + 1]

            for column in curent_sample_df.columns[3:-4]:
                
                del curent_sample_df[column]
            
            del curent_sample_df[curent_sample_df.columns[0]]
            del curent_sample_df[curent_sample_df.columns[0]]

            STE_labels = curent_sample_df[curent_sample_df.columns[0]].to_list()
            STE_labels = transform_strings(STE_labels)
            STE_labels, _ = sort_strings_list(STE_labels)
            
            curent_sample_df[curent_sample_df.columns[0]] = STE_labels
            #del curent_sample_df[curent_sample_df.columns[0]]
            curent_sample_df.columns = ["Наименование нефинансового актива", "Балансовая стоимость", "Количество", "Сумма амортизации", "Остаточная стоимость"]
        
        data_collection[report_type][curent_sample_date] = {
            "data": curent_sample_df,
            "STE_labels": STE_labels
        }
    
    return data_collection


reminder_data_collection = collect_data_reminder(data_path=reminder_data_path)
            

In [9]:
reminder_data_collection["Ведомость остатков (сч. 101)"]["30.06.2022"]["data"].iloc[:100].tail()

,Наименование нефинансового актива,Балансовая стоимость,Количество,Сумма амортизации,Остаточная стоимость
95,волс нижний переулок система строение сусальны...,150401.22,1,150401.22,0
96,помещение система скуд этаж яковоапостольский,68456.14,1,68456.14,0
97,пресненская система скуд этаж,431675.1,1,46250.91,385424.19
98,видеонаблюдения пресненская система этаж,587877.1,1,62986.86,524890.24
99,дозиметр радиометр,41551.2,1,41551.2,0


In [10]:
reminder_data_collection["Ведомость остатков (сч. 105)"]["30.06.2022"]["data"].iloc[:100].tail()

,МОЛ,Цена,Количество,Сумма
95,torsion насадка шуруповерта,19.04,16.0,304.64
96,daikin nord кондиционер настенный пульт,267641.64,1.0,267641.64
97,retractable stenley кровельный лезвием трапеци...,452.08,1.0,452.08
98,sparta отвертка тестер,259.35,2.0,518.70
99,монтажной очиститель пены,208.75,2.0,417.50


In [6]:
trade_inf_ph = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\Выгрузка контрактов по Заказчику.xlsx"
cll_path = "C:\\Users\\1\\Desktop\\LDTProject\\dataset\\КПГЗ ,СПГЗ, СТЕ.xlsx"
trade_inf_dataframe = pd.read_excel(trade_inf_ph)
cll_dataframe = pd.read_excel(cll_path)

Формирование информации о закупках пользователя за все время с файла "Выгрузка контрактов по заказчику"

In [7]:
STE_collection = []
cn_collection = {}

for ts_number in range(trade_inf_dataframe.shape[0]):

    trade_sample = trade_inf_dataframe.iloc[ts_number]
    cll_dataframe_batch = cll_dataframe[(cll_dataframe["Реестровый номер в РК"] == trade_sample["Реестровый номер в РК"])
                                    & (cll_dataframe["КПГЗ код"] == trade_sample["Конечный код КПГЗ"])
                                    & (cll_dataframe["КПГЗ"] == trade_sample["Конечное наименование КПГЗ"])]
    
    trade_sample_STE = cll_dataframe_batch["Название СТЕ"].to_list()
    trade_sample_STE = transform_strings(trade_sample_STE)
    trade_sample_STE, sub_cn_collection = sort_strings_list(trade_sample_STE)
    

    if len(trade_sample_STE) > 2:

        for variant in trade_sample_STE:

            STE_collection.append((variant, 
                                   trade_sample["Конечное наименование КПГЗ"], 
                                   sub_cn_collection[variant]))

    elif len(trade_sample_STE) == 1:
        STE_collection.append((trade_sample_STE[0], 
                               trade_sample["Конечное наименование КПГЗ"], 
                               sub_cn_collection[trade_sample_STE[0]]))
    

In [ ]:
found_reg = {}
found_con = 0

indexator = 0
for trade_sample in STE_collection:

    STE_0 = data_collection["Оборотная ведомость сч. 105"]["general_STE_labels"]
    STE_1 = data_collection["Оборотно-Сальдовая ведомость сч. 101"]["general_STE_labels"]
    STE_2 = data_collection["Оборотно-Сальдовая ведомость сч. 21"]["general_STE_labels"]

    data_labels = ["Оборотная ведомость сч. 105", "Оборотно-Сальдовая ведомость сч. 101", "Оборотно-Сальдовая ведомость сч. 21"]


    for (sample_number, (label_0, label_1, label_2)) in enumerate(zip(STE_0, STE_1, STE_2)):
        
        labels = [label_0, label_1, label_2]
        done_condition = False
        if done_condition:
            break

        for (label_number, label) in enumerate(labels):

            if type(label) != float:
                
                in_label = all([ts_string in label.split() for ts_string in trade_sample[0].split()
                                if (type(label) != float) & (type(trade_sample[0]) != float)])
                if (in_label):
                    
                    found_con += 1
                    serched_dataframe = data_collection[data_labels[label_number]][]
                    found_reg[trade_sample[0]] = {
                        "тип_ведомости": data_labels[label_number],
                        "расположение_в_ведомости": sample_number,
                        "СТЕ наименование": trade_sample[2],
                        "КПГЗ": trade_sample[1]
                        }
                    done_condition = True
                    break
        


with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)

print(found_reg)
print(found_con)

SyntaxError: invalid syntax (1816562737.py, line 30)

In [ ]:
found_reg = {}
found_con = 0

indexator = 0
for trade_sample in STE_collection:

    data_batch_0 = data_collection["Оборотная ведомость сч. 105"]
    data_batch_1 = data_collection["Оборотно-Сальдовая ведомость сч. 101"]
    data_batch_2 = data_collection["Оборотно-Сальдовая ведомость сч. 21"]
    
    for kvartal in data_batch_0.keys():

        if kvartal != "general_STE_labels":

            for (STE_label_number, STE_label) in enumerate(data_batch_0[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотная ведомость сч. 105",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break

            for (STE_label_number, STE_label) in enumerate(data_batch_1[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотно-Сальдовая ведомость сч. 101",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break

            for (STE_label_number, STE_label) in enumerate(data_batch_2[kvartal]["STE_labels"]):

                if type(STE_label) != float:

                    in_label = all([ts_sub_string in STE_label.split() 
                                    for ts_sub_string in trade_sample[0] 
                                    if type(trade_sample[0]) != float])
                    if in_label:
                        
                        found_reg[trade_sample[2]] = {
                            "тим ведомости": "Оборотно-Сальдовая ведомость сч. 21",
                            "КПГЗ": trade_sample[1],
                            f"{kvartal}": {
                                "position in dataframe": STE_label_number
                            }  
                        }
                        break


            
    
        


with open("C:\\Users\\1\\Desktop\\LDTProject\\chek_results.json", "w") as json_file:
    js.dump(found_reg, json_file)

print(found_reg)
print(found_con)

Формирование конечного набора данных в виде json словаря для пследующего анализа информации

In [44]:
#TODO дописать отбор парамтеров для обучения для отчетов типа сч 101 и сч 21

found_reg_test = {}

found_con_test = 0
found_uncon_test = 0

found_con_rem_test = 0
found_uncon_rem_test = 0

time_classification = {
    "квартал_1": "31.03.2022",
    "квартал_2": "30.06.2022",
    "квартал_3": "30.09.2022",
    "квартал_4": "31.12.2022"
}

batch_classification = {
    "Оборотная ведомость сч. 105": "Ведомость остатков (сч. 105)",
    "Оборотно-Сальдовая ведомость сч. 101": "Ведомость остатков (сч. 101)",
    "Оборотно-Сальдовая ведомость сч. 21": "Ведомость остатков (сч. 21)"
}

for trade_sample in STE_collection:

    
    for data_batch in data_collection.keys():

        reminder_report_type = batch_classification[data_batch]
        curent_data_batch = data_collection[data_batch]
        curent_data_batch_rem = reminder_data_collection[reminder_report_type]

        for time_kvartal in curent_data_batch.keys():
            
            
                
            if time_kvartal != "general_STE_labels":
                    
                amount_reg = {}
                reminder_time_kvartal = time_classification[time_kvartal]

                kvartal_dataframe = curent_data_batch[time_kvartal]["data"]
                kvartal_dataframe_reminder = curent_data_batch_rem[reminder_time_kvartal]["data"]
                
                if data_batch == "Оборотная ведомость сч. 105":

                    serched_dataframe = kvartal_dataframe[kvartal_dataframe["Наименование нефинансового актива"] == trade_sample[0]]
                    serched_dataframe_rem = kvartal_dataframe_reminder[kvartal_dataframe_reminder["МОЛ"] == trade_sample[0]]
                
                else:
                    
                    serched_dataframe_amount = kvartal_dataframe[0][kvartal_dataframe[0]["СТЕ"] == trade_sample[0]]
                    serched_dataframe_count = kvartal_dataframe[1][kvartal_dataframe[1]["СТЕ"] == trade_sample[0]]
                    serched_dataframe_rem = kvartal_dataframe_reminder[kvartal_dataframe_reminder["Наименование нефинансового актива"] == trade_sample[0]]

                desired_condition = False
                if data_batch == "Оборотная ведомость сч. 105":
                        
                    if serched_dataframe.shape[0] != 0:
                        
                        found_con_test += 1
                        start_amount = serched_dataframe["Остаток на 1 января 2022 г.|дебет|сумма"].to_numpy()[0]
                        start_count = serched_dataframe["Остаток на 1 января 2022 г.|дебет|кол-во"].to_numpy()[0]

                        if np.isnan(start_amount):
                            
                            start_amount = serched_dataframe["Оборот за 1 квартал 2022 г.|дебет|сумма"].to_numpy()[0]
                            start_count = serched_dataframe["Оборот за 1 квартал 2022 г.|дебет|кол-во"].to_numpy()[0]
                        
                        if np.isnan(start_amount) == False or np.isnan(start_count) == False:

                            desired_condition = True
                            amount_reg["начальная стоимость"] = str(start_amount)
                            amount_reg["начальное кол-во"] = str(start_count)
                    
                else:
                        
                    if serched_dataframe_amount.shape[0] != 0:
                        
                        found_con_test += 1
                        start_amount = serched_dataframe_amount["Сальдо на начало периода|дебет|сумма"].to_numpy()[0]
                        start_count = serched_dataframe_count["Сальдо на начало периода|дебет|кол."].to_numpy()[0]

                        if np.isnan(start_amount) == False or np.isnan(start_count) == False:
                            
                            desired_condition = True
                            amount_reg["начальная стоимость"] = str(start_amount)
                            amount_reg["начальное кол-во"] = str(start_count)
                        
                         
                if data_batch == "Оборотная ведомость сч. 105":
                        
                    
                    if serched_dataframe_rem.shape[0] != 0:
                        
                        found_con_rem_test += 1
                        end_amount = serched_dataframe_rem["Сумма"].to_numpy()[0]
                        end_count = serched_dataframe_rem["Количество"].to_numpy()[0]

                        if np.isnan(end_amount):
                            end_amount = serched_dataframe["Остаток на 31 марта 2022 г.|дебет|сумма"].numpy()[0]  
                            
                        if np.isnan(end_amount) == False or np.isnan(end_count) == False:
                            
                            desired_condition = True
                            amount_reg["конечная стоимость"] = str(end_amount)
                            amount_reg["конечное кол-во"] = str(end_count)
                    
                else:
                    
                    if serched_dataframe_rem.shape[0] != 0:
                        
                        found_con_rem_test += 1
                        end_amount = serched_dataframe_rem["Остаточная стоимость"].to_numpy()[0]
                        end_count = serched_dataframe_rem["Количество"].to_numpy()[0]

                        if end_amount == 0:
                            end_amount = serched_dataframe_rem["Балансовая стоимость"].to_numpy()[0]
                        
                        if (np.isnan(end_amount) == False) or (np.isnan(end_count) == False):
                            
                            desired_condition = True
                            amount_reg["конечная стоимость"] = str(end_amount)
                            amount_reg["конечное кол-во"] = str(end_count)
                        
                        
   
                    elif serched_dataframe.shape[0] != 0:

                        found_uncon_rem_test += 1
                        if data_batch == "Оборотная ведомость сч. 105":

                            end_amount = serched_dataframe["Остаток на 31 марта 2022 г.|дебет|сумма"].to_numpy()[0]
                            end_count = serched_dataframe["Остаток на 31 марта 2022 г.|дебет|кол-во"].to_numpy()[0]
                                
                            if np.isnan(end_amount) == False or np.isnan(end_count) == False:

                                desired_condition = True
                                amount_reg["конечная стоимость"] = str(end_amount)
                                amount_reg["конечное кол-во"] = str(end_count)
                    
                    

                if desired_condition:
                    
                    if "начальная стоимость" in amount_reg.keys() and "начальное кол-во" in amount_reg.keys():

                        if "конечная стоимость" not in amount_reg.keys():
                            amount_reg["конечная стоимость"] = "0.0"

                        if "конечное кол-во" not in amount_reg.keys():
                            amount_reg["конечное кол-во"] = "0.0"

                        found_reg_test[trade_sample[2]] = {}
                        found_reg_test[trade_sample[2]]["тип ведомости"] = data_batch
                        found_reg_test[trade_sample[2]]["Классификация"] = trade_sample[1]
                        found_reg_test[trade_sample[2]][time_kvartal] = amount_reg



print(found_con_test)
print(found_con_rem_test)

with open("C:\\Users\\1\\Desktop\\LDTProject\\test_results.json", "w") as json_file:
    js.dump(found_reg_test, json_file)

489
368


Анализ и обработка данных

In [46]:
all_types = [found_reg_test[trade_sample]["Классификация"] for trade_sample in found_reg_test.keys()]
print(all_types)

['БЛОКНОТЫ', 'ИНСТРУМЕНТ РУЧНОЙ ПРОЧИЙ', 'КОНВЕРТЫ ПОЧТОВЫЕ', 'МАСТИКИ ГИДРОИЗОЛЯЦИОННЫЕ', 'ОБУВЬ С ЗАЩИТНЫМ МЕТАЛЛИЧЕСКИМ ПОДНОСКОМ', 'ПЕРФОРАТОРЫ', 'РЕСПИРАТОРЫ', 'РЕСПИРАТОРЫ', 'РЕСПИРАТОРЫ', 'РЕСПИРАТОРЫ', 'ХОЛОДИЛЬНИКИ БЫТОВЫЕ', 'ХОЛОДИЛЬНИКИ БЫТОВЫЕ', 'ШКАФЫ АРХИВНЫЕ МЕТАЛЛИЧЕСКИЕ', 'ШКАФЫ АРХИВНЫЕ МЕТАЛЛИЧЕСКИЕ', 'ПАПКИ КАРТОННЫЕ', 'ПАПКИ КАРТОННЫЕ', 'ПАПКИ КАРТОННЫЕ', 'ЛАСТИКИ', 'МАРКЕРЫ', 'МАРКЕРЫ', 'МАРКЕРЫ', 'МАРКЕРЫ', 'ДИВАНЫ', 'ДИВАНЫ', 'ДИВАНЫ', 'ДИВАНЫ', 'ДИВАНЫ', 'ДИВАНЫ', 'ПОСУДА ОДНОРАЗОВАЯ ПЛАСТМАССОВАЯ', 'БЛОКИ БУМАЖНЫЕ ДЛЯ ЗАПИСЕЙ', 'БЛОКИ БУМАЖНЫЕ ДЛЯ ЗАПИСЕЙ', 'ДОСКИ МАГНИТНО-МАРКЕРНЫЕ', 'ЛИНЕЙКИ', 'ОБУВЬ С ЗАЩИТНЫМ МЕТАЛЛИЧЕСКИМ ПОДНОСКОМ', 'ПАПКИ ПЛАСТИКОВЫЕ', 'ПАПКИ ПЛАСТИКОВЫЕ', 'СРЕДСТВА КОРРЕКТИРУЮЩИЕ КАНЦЕЛЯРСКИЕ', 'СТОЛЫ ЖУРНАЛЬНЫЕ', 'СТОЛЫ ПИСЬМЕННЫЕ', 'СТОЛЫ ПИСЬМЕННЫЕ', 'ТЕРМОПОТЫ БЫТОВЫЕ', 'ШАПКИ ЗИМНИЕ ДЛЯ ВЗРОСЛЫХ', 'ШТЕМПЕЛИ', 'ЛЕНТА КЛЕЙКАЯ КАНЦЕЛЯРСКАЯ', 'МЕТИЗЫ, КРЕПЕЖНЫЕ ИЗДЕЛИЯ ПРОЧИЕ', 'МЕТИЗЫ, КРЕПЕЖНЫЕ ИЗДЕЛИЯ ПРОЧИЕ', 'МЕТИЗЫ, КРЕПЕЖНЫЕ ИЗ

In [54]:
def formulate_data(ts_collection_path):

    data_collection = {}
    with open(ts_collection_path, "r") as json_file:

        dataset = js.load(json_file)
        cll_types = [dataset[trade_sample]["Классификация"] for trade_sample in dataset]
        
        for cll_type in cll_types:
            
            sub_collection = {}
            sub_collection["кол-во начало"] = []
            sub_collection["сумма начало"] = []
            sub_collection["название"] = []
            sub_collection["кол-во конец"] = []
            sub_collection["сумма конец"] = []

            for trade_sample in dataset.keys():

                if dataset[trade_sample]["Классификация"] == cll_type:

                    sub_collection["название"].append(trade_sample)
                    for kvartal in dataset[trade_sample].keys():

                        if kvartal not in["тип ведомости", "Классификация"]:
                            
                            kvartal_sample = dataset[trade_sample][kvartal]
                            sub_collection["кол-во начало"].append(float(kvartal_sample["начальное кол-во"]))
                            sub_collection["кол-во конец"].append(float(kvartal_sample["конечное кол-во"]))
                            sub_collection["сумма начало"].append(float(kvartal_sample["начальная стоимость"]))
                            sub_collection["сумма конец"].append(float(kvartal_sample["конечная стоимость"]))
            
                    data_collection[cll_type] = sub_collection
    
    return data_collection


result_collection = formulate_data("C:\\Users\\1\\Desktop\\LDTProject\\test_results.json")

                

            

        

In [56]:
def show_data(collection_name):

    need_collection = result_collection[collection_name]
    start_cost = np.asarray(need_collection["сумма начало"])
    end_cost = np.asarray(need_collection["сумма конец"])
    cost_diff = end_cost - 

64
